In [2]:
import os
%pwd

'c:\\Users\\aamer\\OneDrive\\Desktop\\Projects\\train2\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\aamer\\OneDrive\\Desktop\\Projects\\train2'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_dir : Path 
    unzip_dir : Path


In [5]:
from CNNproject.constants import * 
from CNNproject.utils.common import read_yaml, create_directories

class ConfigurationManager: 
    def __init__ ( self,config_file_path = CONFIG_YAML, params_file_path = PARMAS_YAML):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml (params_file_path)
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig: 
        config = self.config.data_ingestion 
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_url,
            local_dir= config.local_dir,
            unzip_dir= config.unzip_dir)

        return data_ingestion_config 


In [6]:
import os 
import urllib.request as request
import zipfile
from CNNproject import logger
from CNNproject.utils.common import get_size


class DataIngestion: 
    def __init__(self, config: DataIngestionConfig): 
        self.config = config 
    
    def download_file(self):
        if not os.path.exists(self.config.local_dir):
            file_name, headers = request.urlretrieve (
                url = self.config.source_url, 
                filename= self.config.local_dir
            )
            logger.info(f"{file_name} downloaded wiht the following info: \n {headers}")
        else: 
            logger.info(f"{file_name} is already exists of size: {get_size(Path(self.config.local_data_file))}")
        
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_dir, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [7]:
try: 
    config = ConfigurationManager()
    data_ingesion_config= config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingesion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e :
    print(e)

[2025-01-29 01:37:42,525: INFO: common: yaml file config\config.yaml loadded successfully]
[2025-01-29 01:37:42,525: INFO: common: yaml file params.yaml loadded successfully]
[2025-01-29 01:37:42,531: INFO: common: Created directory at: artifacts]
[2025-01-29 01:37:42,532: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-01-29 01:37:45,567: INFO: 3087961911: artifacts/data_ingestion/data.zip downloaded wiht the following info: 
 Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 187E:E64FC:5EDF60:621E8B:67995C37
Accept-Ranges: bytes
Date: Tue, 28 Jan 2025 22:37:44 GMT
Via: 1.1 varnish
X-Served-By: cache-mrs10563-MRS
X-Ca